In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # Lamieux Method For Brain Image Segmentation
 This file is supposed to be read by an IDE which supports Jupyter Notebook editors. As a recomendation, the package [Jupyter](https://marketplace.visualstudio.com/items?itemName=donjayamanne.jupyter) for [Visual Studio Code IDE](https://code.visualstudio.com) is a great option.

In [0]:
from __future__ import print_function

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk
import numpy as np

from myshow import myshow, myshow3d


 Loading data and initial visualization...

In [0]:
img_T1 = sitk.ReadImage("../data/IXI002-Guys-0828-MPRAGESEN_-s256_-0301-00003-000001-01.nii")

# To visualize the labels image in RGB with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)

myshow3d(img_T1)


 ### Step 1: Calculating the Foreground Threshold Level

In [0]:
def OstsuThresh(img):
    otsuFilter = sitk.OtsuThresholdImageFilter()
    otsuFilter.SetOutsideValue(255)
    otsuFilter.SetInsideValue(0)
    mask = otsuFilter.Execute(img)
    return mask, otsuFilter.GetThreshold()



In [0]:
mask, threshOtsuValue = OstsuThresh(img_T1)
I_scf = threshOtsuValue / 3.5 + 6
# To visualize the labels image in RGB with needs a image with 0-255 range
myshow(sitk.LabelOverlay(img_T1_255, mask), "Otsu Thresholding")
print('Threshold Otshu value: ', threshOtsuValue)
print('Estimated cerebrospinal fluid (CSF) intensity : ', I_scf)


 ## Step 2 : Initial Brain Disconnection
 Performs a loop where each connected component volume found on the image is calculated.

In [0]:

def Threshold(img, threshValue):
    thresh = sitk.BinaryThresholdImageFilter()
    # defines threshold window
    thresh.SetUpperThreshold(threshValue)
    thresh.SetLowerThreshold(0)
    # defines inner and outer values
    thresh.SetOutsideValue(255)
    thresh.SetInsideValue(0)
    # apply threshold
    return thresh.Execute(img)

def InitialBrainDisconnection(img, threshOtsuValue, threshMult):
    # Compute a mask based on ostu value
    threshValue = threshMult * threshOtsuValue / 3
    mask = Threshold(img, threshValue)
    # Connectivity Components Analysis
    filter = sitk.ConnectedComponentImageFilter()
    components = filter.Execute(img, mask, False)
    # reorder componends
    filter = sitk.RelabelComponentImageFilter()
    reorderedComponents = filter.Execute(components)
    # get largest component size
    component_size = filter.GetSizeOfObjectsInPixels()[0]
    # get image largest component label (pixel value = 1)
    dst = sitk.BinaryThreshold(reorderedComponents, 1, 1)
    return dst, component_size



In [0]:
# significant reduction fo 0.5
significantReduction = 0.5
# initialize variables
threshMult = 1
largestComponentSize = 0
# first call of the interactive function
largestComponentImg, largestComponentSize = InitialBrainDisconnection(img_T1, threshOtsuValue, threshMult)
# initialize loop variables
lastComponetSize = largestComponentSize
MAX_INTERACTIONS = 20 # max interactions of the loop
''' Loops InitialBrainDisconnection ultil a significant reduction of brain volume is observed '''
while(largestComponentSize > (significantReduction * lastComponetSize) and threshMult < MAX_INTERACTIONS):
    # updates loop break condition variables
    threshMult = threshMult + 1
    lastComponetSize = largestComponentSize
    # calls initial brain disconnection
    largestComponentImg, largestComponentSize = InitialBrainDisconnection(img_T1, threshOtsuValue, threshMult)

print('Number of interactions: ', threshMult)
myshow(sitk.LabelOverlay(img_T1_255, largestComponentImg), "Otsu Thresholding")